In [152]:
import utils
import copy
import numpy


# Reprezentarea problemei



In [153]:
class Teacher:
    def __init__(self, name, teacher_info):
        self.name = name
        self.preferred_days = set()
        self.preferred_intervals = set()
        self.courses = set()
        self.__days = set(['Luni', 'Marti', 'Miercuri', 'Joi', 'Vineri'])
        self.interval_count = 0

        for constraint in teacher_info['Constrangeri']:
            # Skip negative constraints, if a day/interval is not in the
            # positive constraint list, it is a negative constraint by default
            if constraint[0] == '!':
                continue

            if constraint in self.__days:
                self.preferred_days.add(constraint)
            else:
                two_hour_intervals = self.__get_two_hour_intervals_from_interval(constraint)
                self.preferred_intervals.update(two_hour_intervals)

        self.courses.update(teacher_info[utils.MATERII])

    def __str__(self):
        return f"{self.name} teaches {self.courses} and prefers {self.preferred_days} and {self.preferred_intervals}"

    def __get_two_hour_intervals_from_interval(self, interval):
        # Evaluate interval as a tuple
        start, end = eval(interval.replace('-', ','))

        result = []

        while start != end:
            result.append((start, start + 2))
            start += 2

        return result

class Course:
    def __init__(self, name, student_count):
        self.name = name
        self.student_count = student_count
        self.students_not_allocated = student_count
        self.teachers = set()
        self.classrooms = set()

    def add_teacher(self, teacher):
        self.teachers.add(teacher)

    def add_classroom(self, classroom):
        self.classrooms.add(classroom)

    def __str__(self):
        return f"{self.name} with {self.student_count} students, teachers {(list(map(lambda t: t.name, self.teachers)))} and classrooms {(list(map(lambda c: c.name, self.classrooms)))}"

class Classroom:
    def __init__(self, name, capacity, courses):
        self.name = name
        self.capacity = capacity
        self.courses = courses

    def __str__(self):
        return f"{self.name} with capacity {self.capacity} and courses {self.courses}"

In [154]:
def parse_yaml(yaml_path):
    yaml_dict = utils.read_yaml_file(yaml_path)
    courses_yaml = yaml_dict[utils.MATERII]
    classrooms_yaml = yaml_dict[utils.SALI]
    teachers_yaml = yaml_dict[utils.PROFESORI]

    teachers = dict()
    courses = dict()
    courses_not_allocated = dict()
    classrooms = dict()

    # Initialize courses
    for course_name, student_count in courses_yaml.items():
        course = Course(course_name, student_count)
        courses[course_name] = course
        courses_not_allocated[course_name] = student_count

    # Parse classroom info
    for classroom_name, classroom_info in classrooms_yaml.items():
        classroom = Classroom(classroom_name, classroom_info['Capacitate'], classroom_info[utils.MATERII])
        classrooms[classroom_name] = classroom
        for course in classroom_info[utils.MATERII]:
            courses[course].add_classroom(classroom)

    # Parse teacher info
    for teacher_name, teacher_info in teachers_yaml.items():
        teacher = Teacher(teacher_name, teacher_info)
        teachers[teacher_name] = teacher
        for course in teacher_info[utils.MATERII]:
            courses[course].add_teacher(teacher)

    return teachers, courses, courses_not_allocated, classrooms

teachers, courses, courses_not_allocated, classrooms = None, None, None, None

In [155]:
class State:
    """ Class that represents a partial timetable allocation """
    def __init__(self, input_yaml_path = None):
        self.timetable = dict()
        self.courses_not_allocated = dict()
        self.conflict_count = 0

        if input_yaml_path is not None:
            self.__parse_yaml_dict(utils.read_yaml_file(input_yaml_path))
            self.courses_not_allocated = copy.deepcopy(courses_not_allocated)


    def __parse_yaml_dict(self, yaml_dict):
        # Initialize timetable with empty classrooms
        self.timetable.update({
            day: {
                eval(interval): {
                    classroom: None
                    for classroom in yaml_dict[utils.SALI]
                }
                for interval in yaml_dict[utils.INTERVALE]
            }
            for day in yaml_dict[utils.ZILE]
        })

    def is_final(self):
        return len(self.courses_not_allocated) == 0

    def clone(self):
        state = State()
        state.timetable = copy.deepcopy(self.timetable)
        state.courses_not_allocated = copy.deepcopy(self.courses_not_allocated)
        state.conflict_count = self.conflict_count

        return state

    def fill_slot(self, day, interval, classroom, teacher_name, course_name):
        new_state = self.clone()
        new_state.timetable[day][interval][classroom] = (teacher_name, course_name)


        teacher = teachers[teacher_name]

        # Update the conflict count
        if day not in teacher.preferred_days:
            new_state.conflict_count += 1

        if interval not in teacher.preferred_intervals:
            new_state.conflict_count += 1

        # Update the course not allocated count
        new_state.courses_not_allocated[course_name] -= courses[course_name].student_count

        return new_state

teachers, courses, courses_not_allocated, classrooms = parse_yaml("inputs/dummy.yaml")
state = State("inputs/dummy.yaml")

new_state = state.fill_slot('Luni', (8, 10), 'EG324', 'Andreea Dinu', 'IA')

print(utils.pretty_print_timetable(new_state.timetable))
print(courses_not_allocated)
print((list(map(str, teachers.values()))))
print((list(map(str, courses.values()))))
print((list(map(str, classrooms.values()))))


|           Interval           |             Luni             |             Marti            |           Miercuri           |              Joi             |            Vineri            |
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|            8 - 10            |      IA : (EG324 - AD)       |      EG324 - goala           |      EG324 - goala           |
|                              |      EG390 - goala           |      EG390 - goala           |      EG390 - goala           |
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|            10 - 12           |      EG324 - goala           |      EG324 - goala           |      EG324 - goala           |
|                              |      EG390 - goala       